In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle
import os
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [2]:
# Load the Datasets
data=pd.read_csv("../Datasets/Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# check data size
data.shape

(10000, 14)

In [4]:
data.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [5]:
# preprocess the data drop unnecessary columns
data.drop(columns=['RowNumber','CustomerId','Surname'],inplace=True)

In [6]:
data.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited'],
      dtype='object')

In [7]:
## check the data type
data.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [8]:
# check the data info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [9]:
# separate the categorical and numerical data 
categorical_columns=data.columns[data.dtypes=='object']
numerical_columns=data.columns[data.dtypes!='object']

In [10]:
categorical_columns

Index(['Geography', 'Gender'], dtype='object')

In [11]:
numerical_columns

Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [12]:
# check the null value in the categorical columnns
categorical_columns.isnull().sum()

0

In [13]:
# check the null value in the numerical columns
numerical_columns.isnull().sum()

0

No null value are available in the categorical or numerical columns

In [14]:
# separte the dependent and independent variable
X=data.drop(columns=['Exited'],axis=1)
y=data['Exited']


In [15]:
# update categorical column and numerical column only independent variable
# mean remove dependent variable target value
categorical_columns=X.select_dtypes(include=['object']).columns
numerical_columns=X.select_dtypes(exclude=['object']).columns



In [16]:
# check the categorical columns
categorical_columns

Index(['Geography', 'Gender'], dtype='object')

In [17]:
# check the numerical columns
numerical_columns

Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary'],
      dtype='object')

In [18]:
# convert categorical columns into numerical columns and convert apply
# standscaler on the numerical columns

preprocessor=ColumnTransformer(
    transformers=[
        ('cate',OneHotEncoder(drop='first',sparse_output=False),categorical_columns),
        ('num',StandardScaler(),numerical_columns)
    ]  
)

# build the pipeline
pipeline =Pipeline(
    steps=[('preprocessor',preprocessor)]
)


# Fit-transformer only feature
X_trasformed =pipeline.fit_transform(X)

print(X_trasformed)

[[ 0.          0.          0.         ...  0.64609167  0.97024255
   0.02188649]
 [ 0.          1.          0.         ... -1.54776799  0.97024255
   0.21653375]
 [ 0.          0.          0.         ...  0.64609167 -1.03067011
   0.2406869 ]
 ...
 [ 0.          0.          0.         ... -1.54776799  0.97024255
  -1.00864308]
 [ 1.          0.          1.         ...  0.64609167 -1.03067011
  -0.12523071]
 [ 0.          0.          0.         ...  0.64609167 -1.03067011
  -1.07636976]]


In [19]:
# check the final data

ohe = pipeline.named_steps['preprocessor'].named_transformers_['cate']
encoded_cat_cols = ohe.get_feature_names_out(categorical_columns)

all_columns = list(encoded_cat_cols) + list(numerical_columns)

# ✅ Convert back to DataFrame
X_df = pd.DataFrame(X_trasformed, columns=all_columns, index=X.index)

print(X_df.head())

   Geography_Germany  Geography_Spain  Gender_Male  CreditScore       Age  \
0                0.0              0.0          0.0    -0.326221  0.293517   
1                0.0              1.0          0.0    -0.440036  0.198164   
2                0.0              0.0          0.0    -1.536794  0.293517   
3                0.0              0.0          0.0     0.501521  0.007457   
4                0.0              1.0          0.0     2.063884  0.388871   

     Tenure   Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0 -1.041760 -1.225848      -0.911583   0.646092        0.970243   
1 -1.387538  0.117350      -0.911583  -1.547768        0.970243   
2  1.032908  1.333053       2.527057   0.646092       -1.030670   
3 -1.387538 -1.225848       0.807737  -1.547768       -1.030670   
4 -1.041760  0.785728      -0.911583   0.646092        0.970243   

   EstimatedSalary  
0         0.021886  
1         0.216534  
2         0.240687  
3        -0.108918  
4        -0.365276  


In [ ]:
# split the training and testing size
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=2)

In [36]:
# fit pip line on the train data
X_train_transformed=pipeline.fit_transform(X_train)
X_test_transformed=pipeline.transform(X_test)

In [64]:
# save the preprocessing pipeline model
with open("../model/model_pipeline.pkl","wb") as f:
    pickle.dump(pipeline,f)

In [21]:
## TraineANN
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [41]:
### Building the model
model=Sequential(
    [Dense(64,activation='relu',input_shape=(X_train_transformed.shape[1],)),
     Dense(32,activation='relu'),
     Dense(1,activation='sigmoid')
    ]
)

In [42]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                768       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2881 (11.25 KB)
Trainable params: 2881 (11.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [43]:
### import optimizer
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

In [44]:
## for enhance the accuracy of model need to do forward and backword propogation
## due to this need to compile the model
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [45]:
## create the log directory for store the log
from tensorflow.keras.callbacks import TensorBoard
log_dir="logs/fit"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [54]:
## Setup for Early stoping
early_stoping_callback=EarlyStopping(monitor='val_loss',patience=20,restore_best_weights=True)

In [55]:
## Training the model
history=model.fit(
    X_train_transformed,y_train,validation_data=(X_test_transformed,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stoping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3126 - accuracy: 0.8708 - val_loss: 0.3588 - val_accuracy: 0.8495
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3114 - accuracy: 0.8714 - val_loss: 0.3669 - val_accuracy: 0.8470
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3118 - accuracy: 0.8687 - val_loss: 0.3575 - val_accuracy: 0.8515
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3082 - accuracy: 0.8737 - val_loss: 0.3724 - val_accuracy: 0.8425
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3062 - accuracy: 0.8746 - val_loss: 0.3669 - val_accuracy: 0.8495
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3040 - accuracy: 0.8749 - val_loss: 0.3751 - val_accuracy: 0.8500
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3015 - accuracy: 0.8750 - val_loss: 0.3656 - val_accuracy: 0.8530

In [39]:
X_train_transformed

array([[ 0.        ,  0.        ,  0.        , ...,  0.64920267,
         0.96510833, -0.62027126],
       [ 1.        ,  0.        ,  0.        , ...,  0.64920267,
         0.96510833,  0.54659799],
       [ 0.        ,  0.        ,  0.        , ...,  0.64920267,
         0.96510833, -0.01320771],
       ...,
       [ 0.        ,  0.        ,  1.        , ...,  0.64920267,
        -1.03615311,  1.64356719],
       [ 1.        ,  0.        ,  1.        , ...,  0.64920267,
        -1.03615311,  1.32474442],
       [ 0.        ,  0.        ,  0.        , ...,  0.64920267,
        -1.03615311,  0.88413017]])

In [56]:
# save model file
model.save("../model/Annmodel.h5")

c:\Users\asus\Documents\code\DataScience\Udemy\Generative_AI\GenAI_Work\NLP_in_DL_Pt\ANN_Pt_Im_1\Annenv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [58]:
# visulize log of data
## Load tensor board Extension
%load_ext tensorboard

In [63]:
%tensorboard --logdir Notebooks/logs/fit20250804-012502